In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import requests
# try-except is a fix for Quantum Mobile release v19.03.0 that does not have requests_cache installed
try:
    import requests_cache
except ImportError:
    pass
import json
import urllib.parse as urlparse
import ipywidgets as ipw
from app import AppBase
from glob import glob
from importlib import import_module
from IPython.display import clear_output, Markdown
from IPython.lib import backgroundjobs as bg
from os import path, getenv
from pprint import pformat
from markdown import markdown

In [ ]:
CONFIG_FN = ".launcher.json"
def read_config():
    if path.exists(CONFIG_FN):
        return json.load(open(CONFIG_FN,'r'))
    else:
        return {'order':[], 'hidden':[]} #default config

def write_config(config):
    json.dump(config, open(CONFIG_FN,'w'), indent=2)

In [ ]:
def move_updown(name, delta):
    config = read_config()
    order = config['order']
    n = len(order)
    i = order.index(name)
    del(order[i])
    j = min(n-1, max(0, i + delta))
    order.insert(j, name)
    config['order'] = order
    write_config(config)

In [ ]:
parsed_url = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)
if 'move_up' in parsed_url:
    move_updown(parsed_url['move_up'][0], -1)
elif 'move_down' in parsed_url:
    move_updown(parsed_url['move_down'][0], +1)

In [ ]:
%%javascript
// in case an app was moved up or down
// I need to remove the corresponding variable from the url
// othervise if I update page again the same move will happen

var url = window.location.toString();
function getQueryVariable(variable)
{
       var query = window.location.search.substring(1);
       var vars = query.split("&");
       for (var i=0;i<vars.length;i++) {
               var pair = vars[i].split("=");
               if(pair[0] == variable){return pair[1];}
       }
       return(false);
}
if (name = getQueryVariable("move_up")){
    url = url.replace("move_up=".concat(name), "");
} else if (name = getQueryVariable("move_down")) {
    url = url.replace("move_down=".concat(name), "");
}
window.history.pushState("", "", url);

In [ ]:
from config import aiidalab_home, aiidalab_apps

In [ ]:
def update_cache():
    """Run this process asynchronously"""
    requests_cache.install_cache(cache_name='apps_meta', backend='sqlite', expire_after=3600, old_data_on_error=True)
    requests.get(registry_url)
    requests_cache.install_cache(cache_name='apps_meta', backend='sqlite')

# try-except is a fix for Quantum Mobile release v19.03.0 that does not have requests_cache installed
try:
    requests_cache.install_cache(cache_name='apps_meta', backend='sqlite') # at start getting data from cache
    update_cache_background = bg.BackgroundJobFunc(update_cache) # if requests_cache is installed, the
                                                                 # update_cache_background variable will be present
except NameError:
    pass
registry_url = 'https://aiidalab.materialscloud.org/appsdata/apps_meta.json'
try:
    app_registry = requests.get(registry_url).json()['apps']
    if 'update_cache_background' in globals():
        update_cache_background.start()
except ValueError:
    print("Registry server is unavailable! Can't check for the updates")
    app_registry = {}

In [ ]:
def render():
    clear_output()
    render_home()
    apps = load_apps()
    config = read_config()
    for name in apps:
        accordion = mk_accordion(name)
        accordion.selected_index = None if name in config['hidden'] else 0
        display(accordion)

In [ ]:
def render_home():
    launcher = load_widget('home')
    launcher.layout = ipw.Layout(width="900px", padding="20px", color='gray')
    app = AppBase('home', app_registry.get('home', None), aiidalab_apps)
    update_info = ipw.HTML("{}".format(app.update_info))
    update_info.layout.margin = "0px 0px 0px 800px"
    description_box = ipw.HTML("<a href=./single_app.ipynb?app=home><button>Manage App</button></a> {}".format(
        app.git_hidden_url), layout={'width': 'initial'})
    description_box.layout.margin = "0px 0px 0px 700px"
    info_line = app.install_info
    display(update_info, launcher, description_box, info_line)

In [ ]:
def mk_accordion(name):
    launcher = load_widget(name)
    launcher.layout = ipw.Layout(width="900px")
    btn_box = mk_buttons(name)
    app_data = app_registry.get(name, None)
    app = AppBase(name, app_data, aiidalab_apps)
    update_info = ipw.HTML("{}".format(app.update_info))
    update_info.layout.margin = "0px 0px 0px 800px"
    run_line = ipw.HBox([launcher, btn_box])
    description_box = ipw.HTML("<a href=./single_app.ipynb?app={}><button>Manage App</button></a> {}".format(name, app.git_hidden_url),
                               layout={'width': 'initial'})
    info_line = app.install_info
    description_box.layout.margin = "0px 0px 0px 700px"
    box = ipw.VBox([update_info, run_line, description_box])
    accordion = ipw.Accordion(children=[box])
    title = load_title(name)
    accordion.set_title(0, title)
    on_change = lambda c: record_showhide(name, accordion.selected_index==0)
    accordion.observe(on_change, names="selected_index")
    return accordion

In [ ]:
def load_title(name):
    try:
        fn = path.join(aiidalab_apps, name, 'metadata.json')
        metadata = json.load(open(fn))
        title = metadata['title']
    except:
        title = "%s (couldn't load title)"%name
    return title

In [ ]:
def load_apps():
    apps = [path.basename(fn) for fn in glob(path.join(aiidalab_apps, '*')) if path.isdir(fn) and not fn.endswith('home') and
           not fn.endswith('__pycache__')]
    config = read_config()
    order = config['order']
    apps.sort(key=lambda x: order.index(x) if x in order else -1)
    config['order'] = apps
    write_config(config)
    return apps

In [ ]:
def load_widget(name):
    if path.exists(path.join(aiidalab_apps, name, 'start.py')):
        return load_start_py(name)
    else:  # fall back
        return load_start_md(name)

In [ ]:
def load_start_py(name):
    try:
        mod = import_module('apps.%s.start' % name)
        appbase = "../" + name
        jupbase = "../../.."
        notebase = jupbase+"/notebooks/apps/"+name
        try:
            return mod.get_start_widget(appbase=appbase, jupbase=jupbase, notebase=notebase)
        except:
            return mod.get_start_widget(appbase=appbase, jupbase=jupbase)
    except Exception as e:
        return ipw.HTML("<pre>%s</pre>" % str(e))

In [ ]:
def load_start_md(name):
    fn = path.join(aiidalab_apps, name, 'start.md')
    try:
        
        md_src = open(fn).read()
        md_src = md_src.replace("](./", "](../%s/"%name)
        html = markdown(md_src)

        # open links in new window/tab
        html = html.replace('<a ', '<a target="_blank" ')

        # downsize headings
        html = html.replace("<h3", "<h4")
        return ipw.HTML(html)

    except Exception as e:
        return ipw.HTML("Could not load start.md")

In [ ]:
def mk_buttons(name):
    layout = ipw.Layout(width="40px")
    btn_box = ipw.HTML("""
    <a href=./start.ipynb?move_up={} title="Move it up"><i class='fa fa-arrow-up' style='color:#337ab7;font-size:2em;' ></i></a>
    <a href=./start.ipynb?move_down={} title="Move it down"><i class='fa fa-arrow-down' style='color:#337ab7;font-size:2em;' ></i></a>
    """.format(name, name), layout=layout)
    btn_box.layout.margin = "50px 0px 0px 0px"

    return(btn_box)

In [ ]:
def record_showhide(name, visible):
    config = read_config()
    hidden = set(config['hidden'])
    if visible:
        hidden.discard(name)
    else:
        hidden.add(name)
    config['hidden'] = list(hidden)
    write_config(config)

In [ ]:
#%load_ext line_profiler
#start the app
#%lprun -f mk_accordion render()
#%timeit render()
render()